In [40]:
import pandas as pd
import numpy as np
import os
import argparse

from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from typing_extensions import override
from openai import AssistantEventHandler

from dataclasses import dataclass
import os
import shutil

In [2]:
CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [52]:
q = "Who was Richard Nixon?"

In [53]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

query_text = q

# Prepare the DB.
embedding_function = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

Unable to find matching results.


In [46]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

Human: 
Answer the question based only on the following context:

Who Is LeBron James? LeBron James is an American basketball player with the Los Angeles Lakers. James first garnered national attention as the top high school basketball player in the country. With his unique combination of size, athleticism and court vision, he became a four-time NBA MVP. After

---

his unique combination of size, athleticism and court vision, he became a four-time NBA MVP. After leading the Miami Heat to titles in 2012 and 2013, James returned to Cleveland and helped the franchise claim its first championship in 2016.

---

School team also earned the top national ranking that year. James would soon emerge as one of the National Basketball Association's leading players.

---

Answer the question based on the above context: Who is Lebron James?



In [70]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
  
assistant = client.beta.assistants.create(
  name="RO-B-OT",
  instructions="You are a helpful, polite, and funny assistant who can help answer questions about a person in a way that highlights only their achievements. You make the person seem like a great fit for a programming job.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-3.5-turbo-0125",
)

In [71]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Who was Richard Nixon?"
)

In [72]:
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="When describing a person, only highlight their achievements.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Richard Nixon was the 37th President of the United States, serving from 1969 to 1974. He is known for several achievements during his presidency, including:

1. Opening diplomatic relations with China in 1972
2. Signing the Strategic Arms Limitation Treaty (SALT)
3. Ending American involvement in the Vietnam War
4. Creating the Environmental Protection Agency (EPA)
5. Establishing the Occupational Safety and Health Administration (OSHA)
6. Implementing desegregation of Southern schools
7. Initiating the "War on Cancer" by signing the National Cancer Act

Nixon's presidency was marked by both domestic and international accomplishments, as well as controversies such as the Watergate scandal, which ultimately led to his resignation in 1974.